# Import related functions

In [1]:
import sys
import cobra
import re
import pandas as pd
sys.path.append(r'./script/')
from ECMpy_function import *
from AutoPACMEN_function import *
import datetime

# Input and output files

In [2]:
# Initial parameter
obj='bio00006'# bio00006 EX_lys_L_e
fluxes_outfile = './analysis/ECMpy_solution_%s_pfba.csv'%obj
use_substrate='EX_glc__D_e'
concentration=10
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 0.5
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
enz_ratio=0.1

# Input files
autopacmen_folder = "./iBsu1147_get_data/"
gap_fill= 'mean'#'mean'#'median'
project_name = "iBsu1147_%s"%gap_fill
organism = "Bacillus subtilis"
create_file(autopacmen_folder)
reaction_gene_path = "./data/reaction_gene_subunit_num.json"
#reaction_gene_path='none'
protein_kcat_database_path = "none"
bigg_metabolites_file = "./data/bigg_models_metabolites.txt"#date:20211008 http://bigg.ucsd.edu/static/namespace/bigg_models_metabolites.txt
brenda_textfile_path = "./data/brenda_download.txt"#date:20211008 https://www.brenda-enzymes.org/brenda_download/file_download.php
sbml_path = "./model/iBsu1147_modify.xml"
reaction_kcat_MW_file="./data/reaction_kcat_MW.csv"
gene_abundance_file='./data/paxdb-abundances-bsu.txt'
file_list=[gene_abundance_file]
EC_max_file='./data/EC_kcat_max.json'

# Output files
brenda_json_path = "%skcat_database_brenda.json"%autopacmen_folder
brenda_output_json_path = "%skcat_database_brenda_for_model.json"%autopacmen_folder
bigg_id_name_mapping_path = "%sbigg_id_name_mapping.json"%autopacmen_folder
sabio_rk_output_path = "%skcat_database_sabio_rk.json"%autopacmen_folder
combined_output_path = "%skcat_database_combined.json"%autopacmen_folder
reaction_kcat_MW_file="%s/reaction_kcat_MW.csv"%autopacmen_folder
uniprot_data_file='%s/%s_protein_id_mass_mapping.json'%(autopacmen_folder,project_name)

modelgene2uniprot_file='./analysis/modelgene2uniprot.txt'
paxdbgene2uniprot_file='./analysis/paxdbgene2uniprot.txt'
ecModel_output_file="./model/iBsu1147_irr_enz_constraint_1102.json"
json_output_file = './model/iBsu1147_irr_enz_constraint_adj.json'
reaction_kcat_MW_outfile = './analysis/reaction_change_by_enzuse.csv'


path exists


# Get reaction kcat_mw using AutoPacmen

## https://github.com/klamt-lab/autopacmen

In [3]:
starttime=datetime.datetime.now()
# Step 1: get bigg metbolite
create_file(autopacmen_folder)
print("Starting to deal BIGG metabolites text file...")
parse_bigg_metabolites_file(bigg_metabolites_file, autopacmen_folder)
print("BIGG metabolites text file done!")
print()

# Step 2: BRENDA kcat
print("Starting to deal BRENDA textfile...")
parse_brenda_textfile(brenda_textfile_path, autopacmen_folder, brenda_json_path)
print("BRENDA textfile done!")
print()

# Step 3: Select Brenda kcat for model
print("Starting to deal brenda json for model...")
parse_brenda_json_for_model(sbml_path, brenda_json_path, brenda_output_json_path)
print("BRENDA json for model done!")
print()

# Step 4: SABIO-RK kcat for model
print("Starting EC numbers kcat search in SABIO-RK...")
parse_sabio_rk_for_model_with_sbml(sbml_path, sabio_rk_output_path, bigg_id_name_mapping_path)
print("SABIO-RK done!")
print()

# Step 5: Brenda and SABIO-RK kcat combined
print("Combining kcat database...")
create_combined_kcat_database(sabio_rk_output_path, brenda_output_json_path, combined_output_path)
print("Combining kcat database done!")
print()

# Step 6: kcat assignment for model
print("Starting to assign kcat for model...")
get_reactions_kcat_mapping(sbml_path, autopacmen_folder, project_name, organism, combined_output_path, protein_kcat_database_path,gap_fill)
print("kcat assignment done!")
print()

# Step 7: get mw for model gene (must be uniprot ID)
print("Starting UniProt ID<->Protein mass search using UniProt API...")
get_protein_mass_mapping_with_sbml(sbml_path, autopacmen_folder, project_name)
print("Protein ID<->Mass mapping done!")

# Step 8: get_reaction_kcat_mw for model
print("Starting to get reaction kcat_mw for model...")
get_reaction_kcat_mw(sbml_path,autopacmen_folder, project_name, reaction_gene_path, gap_fill)       
print("reaction kcat_mw done!")
endtime=datetime.datetime.now()
print(endtime-starttime)

path exists
Starting to deal BIGG metabolites text file...
BIGG metabolites text file done!

Starting to deal BRENDA textfile...
BRENDA textfile done!

Starting to deal brenda json for model...
BRENDA json for model done!

Starting EC numbers kcat search in SABIO-RK...
SABIO-RK done!

Combining kcat database...
Combining kcat database done!

Starting to assign kcat for model...
kcat assignment done!

Starting UniProt ID<->Protein mass search using UniProt API...
Protein ID<->Mass mapping done!
Starting to get reaction kcat_mw for model...
Default kcat is: 5759.84635232689
reaction kcat_mw done!
0:13:52.777492


# Calculate f-value

In [4]:
model=cobra.io.read_sbml_model(sbml_path)
model_gene_list=[]
for eachg in model.genes:
    model_gene_list.append(eachg.id)
uni_model_gene_list=list(set(model_gene_list))

gene_abundance = pd.read_csv(gene_abundance_file, index_col=0,sep='\t')
uni_paxdb_gene_list=list(set(list(gene_abundance.index)))

GENENAME_2_ACC_from_uniprot(uni_model_gene_list,modelgene2uniprot_file)
GENENAME_2_ACC_from_uniprot_byID(uni_paxdb_gene_list,paxdbgene2uniprot_file)

f=calculate_f_special(gene_abundance_file, modelgene2uniprot_file,paxdbgene2uniprot_file,'abundance')
f

0.5881478848125352

# Get ecModel and simulation

In [5]:
#The enzyme mass fraction
f=0.5881478848125352
f=round(f, 3)
print('Enzyme mass fraction: ',f)
upperbound = round(ptot * f * sigma, 3)
print('Enzyme bound: ',upperbound)

trans_model2enz_json_model_split_isoenzyme(sbml_path, reaction_kcat_MW_file, f, ptot, sigma, lowerbound, upperbound, ecModel_output_file)
enz_model=get_enzyme_constraint_model(ecModel_output_file)
enz_model.objective=obj

#Using only the provided substrate
[ori_obj_id,ori_substrate_id_list,ori_sub_concentration,ori_ATPM]=get_model_substrate_obj(enz_model)
for eachsubid in ori_substrate_id_list:
    if re.search('_reverse',eachsubid):
        r_id_new=eachsubid.split('_reverse')[0]
        enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
        enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0)  
    else:
        r_id_new=eachsubid+'_reverse'
        enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
        enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0) 
        
enz_model.reactions.get_by_id(use_substrate).bounds = (-concentration, 0)
enz_model.reactions.get_by_id(use_substrate+'_reverse').bounds = (0, 0)

enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,ecModel_output_file)
print(round(enz_model_pfba_solution.fluxes[obj],3))

Enzyme mass fraction:  0.588
Enzyme bound:  0.165
0.082


# Calibration kcat according Enzyme usage

In [4]:
fluxes_infile_ori = './analysis/ECMpy_solution_%s_pfba.csv'%obj
fluxes_outfile = fluxes_infile_ori
need_change_reaction_list=[]
changed_reaction_list=[]
concentration = 7.63
round_num=1
reaction_kcat_mw = pd.read_csv(reaction_kcat_MW_file, index_col=0)

#Model adjustment
enz_model_obj_max=0
# High-Quality Genome-Scale Reconstruction of Corynebacterium glutamicum ATCC 13032. Front Microbiol. Frontiers Media S.A.; 2021;12:3432. 
# Beyond growth rate 0.6: What drives Corynebacterium glutamicum to higher growth rates in defined medium. Biotechnol Bioeng [Internet]. Biotechnol Bioeng; 2014 [cited 2021 Sep 5];111:359-71. Available from: https://pubmed.ncbi.nlm.nih.gov/23996851/
while (enz_model_obj_max<0.60):

    [enz_model,reaction_kcat_mw,need_change_reaction_list, changed_reaction_list]=change_enz_model_by_enz_usage(enz_ratio,\
                                                    ecModel_output_file,fluxes_outfile,EC_max_file,\
                                                    reaction_kcat_mw,need_change_reaction_list,changed_reaction_list,f, \
                                                    ptot, sigma, lowerbound, upperbound, json_output_file)
    #Using only the provided substrate
    [ori_obj_id,ori_substrate_id_list,ori_sub_concentration,ori_ATPM]=get_model_substrate_obj(enz_model)
    for eachsubid in ori_substrate_id_list:
        print(eachsubid)
        if re.search('_reverse',eachsubid):
            r_id_new=eachsubid.split('_reverse')[0]
            enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
            enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0)  
        else:
            r_id_new=eachsubid+'_reverse'
            enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
            enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0) 
    enz_model.reactions.get_by_id(use_substrate).bounds = (-concentration, 0)
    enz_model.reactions.get_by_id(use_substrate+'_reverse').bounds = (0, 0)

    enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
    fluxes_outfile = './analysis/ECMpy_solution_adjround%s.csv'%(round_num)
    enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,json_output_file)
    
    enz_model_obj=enz_model.slim_optimize()

   #最大生长对应的底物摄取
    obj='bio00006'# CG_biomass_cgl_ATCC13032 EX_lys_L_e EX_trp_L_e
    substrate='EX_glc__D_e'
    enz_model.reactions.get_by_id(substrate).bounds=(-1000,0)
    enz_model.reactions.get_by_id('EX_glc__D_e_reverse').bounds=(0,0)
    # enz_model.reactions.get_by_id('CG_biomass_cgl_ATCC13032').bounds=(0.1,0.1)
    enz_model.objective=obj
    enz_model_solution = cobra.flux_analysis.pfba(enz_model)
    print("ecGEM max glucose uptake rates:"+str(-enz_model_solution.fluxes[substrate]))
    print("ecGEM max growth rates:"+str(enz_model_solution.fluxes[obj]))
    enz_model_sub=-enz_model_solution.fluxes[substrate]
    enz_model_obj_max=enz_model_solution.fluxes[obj]

    print('Round %s: '%round_num)
    round_num=round_num+1
    if round_num>50:
        break
    print()
reaction_kcat_mw.to_csv(reaction_kcat_MW_outfile)

Need changing reaction: 
ATNS_nh4
Changed reaction: 
['ATNS_nh4']
EX_glc__D_e_reverse
ecGEM max glucose uptake rates:4.4753943135732435
ecGEM max growth rates:0.1141241666385906
Round 1: 

Need changing reaction: 
ADK1m
Changed reaction: 
['ATNS_nh4', 'ADK1m']
EX_glc__D_e_reverse
ecGEM max glucose uptake rates:6.473928976378684
ecGEM max growth rates:0.18008124186800148
Round 2: 

Need changing reaction: 
GLUTRS_Gln
Changed reaction: 
['ATNS_nh4', 'ADK1m', 'GLUTRS_Gln']
EX_glc__D_e_reverse
ecGEM max glucose uptake rates:7.127074560552723
ecGEM max growth rates:0.20163690133344672
Round 3: 

Need changing reaction: 
GLUTRS
Changed reaction: 
['ATNS_nh4', 'ADK1m', 'GLUTRS_Gln', 'GLUTRS']
EX_glc__D_e_reverse
ecGEM max glucose uptake rates:7.957844267980703
ecGEM max growth rates:0.2290546613879216
Round 4: 

Need changing reaction: 
ENOm
Changed reaction: 
['ATNS_nh4', 'ADK1m', 'GLUTRS_Gln', 'GLUTRS', 'ENOm']
EX_glc__D_e_reverse
ecGEM max glucose uptake rates:8.31124123404761
ecGEM max gr